## Errors 

- not every songs starts with keysignature, time indication
- keysignature should be added (not necessarily in C when it is missing)


In [ ]:
from music21 import *
import copy

In [ ]:
sa = converter.parse("/Users/jvo/Library/Mobile Documents/com~apple~CloudDocs/bladmuziek/realbooks/Ultime_Jazz_Real_Book_-_A.mxl")
sb = converter.parse("/Users/jvo/Library/Mobile Documents/com~apple~CloudDocs/bladmuziek/realbooks/Ultime_Jazz_Real_Book_-_B.mxl")

In [ ]:
s = sb

In [ ]:
# offsetStarts = [keyStart.getOffsetInHierarchy(s) for keyStart in s[key.KeySignature]]
offsetsStartPage = [0] + [el.getOffsetInHierarchy(s) for el in s[layout.PageLayout]]

In [ ]:
len(offsetsStartPage)

In [ ]:
metadataPerPage = [[page,None,None] for page in range(1,len(offsetsStartPage) + 1)]

for tb in s[text.TextBox]:
    if tb.page > len(offsetsStartPage):
        continue
    
    if tb.style.justify == 'right':
        metadataPerPage[tb.page-1][2] = tb.content
        
    if tb.style.justify == 'center' and not metadataPerPage[tb.page-1][1]:
        metadataPerPage[tb.page-1][1] = tb.content

metadataPerPage

In [ ]:
noStartIdx = []

for i, datum in enumerate(metadataPerPage):
    if not datum[2]:
        noStartIdx.append(i)

offsetsStartSong = [offset for i, offset in enumerate(offsetsStartPage) if i not in noStartIdx]
metadataSongs = [datum for i, datum in enumerate(metadataPerPage) if i not in noStartIdx]
offsetsStartSong


In [ ]:
measureNumbersStartSong = []
measures = s[stream.Measure]
j = 1
for i in range(len(measures)):
    if measures[i].offset in offsetsStartSong:
        measureNumbersStartSong.append(measures[i].number)

#         indexPrevMeasure = max(0,i-1)
#         barlinePrevMeasure = measures[indexPrevMeasure][bar.Barline].last()
#         if barlinePrevMeasure and barlinePrevMeasure.type=='final' or True:
#             measureNumbersStart.append(measures[i].number)

lastMeasureNumber = s[stream.Measure][-1].number
measureNumbersStartEndSong = []

for measureNumberStart, measureNumberStartNext in zip(measureNumbersStartSong, measureNumbersStartSong[1:]):
    measureNumbersStartEndSong.append((measureNumberStart, measureNumberStartNext))

measureNumbersStartEndSong.append((measureNumbersStartSong[-1], lastMeasureNumber))

len(measureNumbersStartEndSong)

In [ ]:
songs = []
for measureNumberStart, measureNumberEnd in measureNumbersStartEndSong:
    songs.append(s.measures(measureNumberStart, measureNumberEnd - 1))

    

In [ ]:
for i, song in enumerate(songs):
        
    song.insert(0, metadata.Metadata())
    song.metadata.title = metadataSongs[i][1]
    song.metadata.composer = metadataSongs[i][2]
    
    song.write("musicxml", f"output-realbook/{song.metadata.title}.musicxml")
